# Getting New Data

In [ ]:
#Josh's code
basegame = 4108071572
url = 'https://americas.api.riotgames.com/'
name = 'NoNoYogi'
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
puuid =response.json()['puuid']
pulls = 0
count = 1
while pulls != 100:
    
    query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={count}&count=1&api_key={api_key}'
    response = requests.get(url+query)
    
    for game in response.json():
        if int(game[4:]) > basegame:
            print("higher")
        else:
            print('lower')
    count +=1
    pulls +=1

In [ ]:
#Jared's code
def get_new_match_ids(puuids):
    url = 'https://americas.api.riotgames.com/'
    for puuid in puuids:
        i = 1
        while i < 20:
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i-1}&count={i}&api_key={api_key}'
            response = requests.get(url+query)
            match = response.json()
            match_num = int(match[0].strip('NA1_'))
            if match_num > 4108071572:
                i += 1
                match_list.append(match)
            else:
                break

### Combine the Code Above Into One Function

In [53]:
def get_new_patch_data(name_list, api_key):
    """
    This function takes in a list of usernames that are in the same tier and iterates 
    through them to find their puuids.
    It will then use those puuids to grab all of their match ids that are in the newest patch.
    From there, the function will iterate through all of the match ids and grab the timeline
    data for each match as well as the other game data. 
    Finally, this function will incrementally save the returned json information every 10 times.
    This prevents the loss of all data if the function errors out at some point.
    This function returns nothing.
    """
    
    #Set up the basegame number. Any match_ids below this number are part of the old patch
    #and will not be included. We will use the basegame var for comparisons in the loop
    basegame = 4107000001
    
    #Set up the base url for grabbing match_ids
    match_url = 'https://americas.api.riotgames.com/'
    
    #Set up the base url for grabbing puuids
    puuid_base_url = 'https://na1.api.riotgames.com/'
    
    #Set up another counter to keep track of how many users have been completed
    n = 0
    
    #Set up a counter to keep track of how many json files have been completed in total
    num_json = 0
    
    #Set up one more counter to keep track of how many pulls we have made.
    #Can't go over 100 pulls every 2 minutes.
    num_pulls = 0
    
    #Set up empty lists to store timeline and other game data
    timeline_data = []
    game_data = []
    
    #Loop through each name in the given name_list
    #First grab the puuid, then grab the match_ids, then actual match json info
    for name in name_list:
        
        #for testing
        print(f'Beginning Loop for user {name}')
        
        #Update n var
        n += 1
        
        #Leave a status update after every 100 users
        if n % 100 == 0:
            print(f'Completed {n} users of {len(name_list)}!\n')
        
        ########################## Get puuids ##########################
        
        #Set up the query
        query = f'lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
        
        #Get the response
        response = requests.get(puuid_base_url + query)
        
        #Update pull count and sleep for 150 seconds if necessary
        num_pulls += 1
        
        if num_pulls % 100 == 0:
            print('\nHit 100 pulls! Sleeping for 150 seconds.')
            sleep(150)
            print('Continuing!\n')
        
        #Check the response code and leave message if something goes wrong
        if response.status_code != 200:
            print(f'Something went wrong! Status Code: {response.status_code}. Did not get puuid for user: {name}')
            print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
            continue
        
        #Grab the user's puuid
        puuid = response.json()['puuid']
        
        #For testing
        print(f'\t-Got the puuid for user {name}.')
        
        ########################## Get Match IDs #########################
        
        #Now that we have a single puuid, grab all the match ids for that user that are greater
        #than the basegame value. These match_ids will be for the newest patch.
        
        #Set up a counter
        i = 0
        
        #This loop will be set up to run indefinitely. Be sure to break out of it when 
        #Done with the current user
        while i >= 0:
            #Set up the query to get a match_id
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i}&count=1&api_key={api_key}'
            
            #Get the response
            response = requests.get(match_url + query)
            
            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response status code and leave message if something went wrong
            if response.status_code != 200:
                print(f'Something went wrong! Status Code: {response.status_code}. Did not get Match_ID for user: {name}')
                print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
                break
                
            #Grab the match_id
            match = response.json()
            
            #Verify there's something in it
            if len(match) == 0:
                print(f'\t-No match ID for user {name}. Skipping this user!')
                break
            
            #Convert match_id to integer
            match_num = int(match[0].strip('NA1_'))
            
            #Compare current match_id to basegame var.
            #Skip if match_id is less than basegame
            if match_num < basegame:
                print(f'\t-Completed getting all current patch Match IDs for user {name}')
                print(f'\t-There are {len(name_list) - n} users left.')
                break

            ####################### Get Match Info ##########################
            
            #Now that we know the current match_id is part of the newest patch,
            #We can retrieve the actual match info. Keep in mind this is still part
            #of the while loop above
                
            #Every 500 entries, save the lists as json files and reset the lists
            if len(timeline_data) % 10 == 0:    
                #Save timeline data.
                #First, convert it to a df
                timeline_df = pd.DataFrame(timeline_data)

                #Save as json file. Use username string to identify whose file it is
                timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created timeline_data_user_{name}_end_{num_json}.json')

                #Save other game data
                #First, convert it to a df
                game_df = pd.DataFrame(game_data)

                #Save as json file. Use username string to identify whose file it is
                game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created other_game_data_user_{name}_end_{num_json}.json\n')
                
                #Now reset the temp lists
                timeline_data = []
                game_data = []
            
        
            ############################# TIMELINE DATA #######################
            #Set up timeline url
            timeline_query = f'lol/match/v5/matches/{match[0]}/timeline/?api_key={api_key}'

            #Grab timeline json data
            timeline_response = requests.get(match_url + timeline_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message
            if timeline_response.status_code != 200:
                print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID.\n')
                continue

            #Turn it into json format
            timeline_json = timeline_response.json()

            #Append this data to the timeline_data list
            timeline_data.append(timeline_json)
            
            ############################# OTHER GAME DATA #######################
            #Set up game data url
            game_query = f'lol/match/v5/matches/{match[0]}?api_key={api_key}'

            #Grab game json data
            game_response = requests.get(match_url + game_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message.
            if game_response.status_code != 200:
                print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')

                #Remove the last entry in the timeline_data list
                timeline_data.pop()
                continue

            #Turn it into json format
            game_json = game_response.json()

            #Append this data to the game_data list
            game_data.append(game_json)
            
            #Update num_json
            num_json += 1
            
            #Update i
            i += 1
            
        #Tell the user how many match_ids were used for the current user
        print(f'\t-Completed {i} Match IDs for user {name}\n')
            
    #Complete! Leave a message
    print('All users completed! Follow suit.')
    
    #Return any json data that was not saved at the end of the function
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)

    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created timeline_data_user_{name}_end_{num_json}.json')

    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)

    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created other_game_data_user_{name}_end_{num_json}.json')
    
    #For testing
    return timeline_data, game_data
            

In [ ]:
#For testing, import the user name
from pro_names import pro_names
from env import api_key
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key
from time import sleep

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [54]:
test_list = pro_names[:10]
test_list

['Huni',
 'CoreJJ',
 'Impact',
 'Contractz',
 'Jensen',
 'Sneaky',
 'Smoothie',
 'Westrice',
 'Darshan',
 'Dardoch']

In [55]:
last_timeline_data, last_game_data = get_new_patch_data(test_list, api_key)

Beginning Loop for user Huni
	-Got the puuid for user Huni.
	-Completed getting all current patch Match IDs for user Huni
	-There are 9 users left.
	-Completed 0 Match IDs for user Huni

Beginning Loop for user CoreJJ
	-Got the puuid for user CoreJJ.
	-Completed getting all current patch Match IDs for user CoreJJ
	-There are 8 users left.
	-Completed 0 Match IDs for user CoreJJ

Beginning Loop for user Impact
	-Got the puuid for user Impact.
	-Completed getting all current patch Match IDs for user Impact
	-There are 7 users left.
	-Completed 0 Match IDs for user Impact

Beginning Loop for user Contractz
	-Got the puuid for user Contractz.
	-Completed getting all current patch Match IDs for user Contractz
	-There are 6 users left.
	-Completed 0 Match IDs for user Contractz

Beginning Loop for user Jensen
	-Got the puuid for user Jensen.
	-Created timeline_data_user_Jensen_end_0.json
	-Created other_game_data_user_Jensen_end_0.json

	-Created timeline_data_user_Jensen_end_10.json
	-Creat

In [56]:
len(last_timeline_data)

3

In [57]:
timeline_data = pd.read_json('timeline_data_user_Jensen_end_10.json')
game_data = pd.read_json('other_game_data_user_Jensen_end_10.json')

In [58]:
#Convert to list of dicts
timeline_data = timeline_data.to_dict(orient = 'records')
game_data = game_data.to_dict(orient = 'records')

In [59]:
import prepare

In [60]:
df = prepare.prepare(timeline_data, game_data, 20)

Finished with: 0 of 10
Finished with: 1 of 10
Finished with: 2 of 10
Finished with: 3 of 10
Skipping: 4 due to <20 min or not classic
Finished with: 5 of 10
Finished with: 6 of 10
Skipping: 7 due to <20 min or not classic
Finished with: 8 of 10
Finished with: 9 of 10


In [64]:
df = prepare.prep(df)

In [66]:
df.shape

(8, 207)

In [68]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 207 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    killsplayer_1                          float64
 1    killsplayer_2                          float64
 2    killsplayer_4                          float64
 3    killsplayer_5                          float64
 4    killsplayer_6                          float64
 5    killsplayer_7                          float64
 6    killsplayer_8                          float64
 7    killsplayer_9                          float64
 8    killsplayer_10                         float64
 9    deathsplayer_1                         float64
 10   deathsplayer_2                         float64
 11   deathsplayer_3                         float64
 12   deathsplayer_4                         float64
 13   deathsplayer_5                         float64
 14   deathsplayer_6                         float